# Benchmark load_by_run_spec

The time required to load a dataset using `load_by_run_spec`when compared to the time taken by `load_by_ guid` or `load_by_id` is two orders of magnitude higher for a large database, in this case 10 gb. 

Once the databased is "vacuum"ed, the time to retrieve dataset decreases by 4 times. However, it takes around 10min to vacuum a database each time you do it. This is irrespective of the fact if you just vacuumed your db. 

The time to load dataset using `load_by_run_spec` comes at par with other methods once the proper indexing is done. Also, the total time to upgrade the database is just around 10 sec.  

It was not possible to lower the search time from 300ms because, as Jens have mentioned below, the connection to db is significantly slower for larger db. In this case connection time is around 250 ms. Time for actual search is hence only few ms. 

In [1]:
from qcodes import initialise_or_create_database_at, load_by_id, load_by_guid, load_by_run_spec

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-halakh\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-halakh\.qcodes\logs\200204-10220-qcodes.log


In [2]:
initialise_or_create_database_at(r"C:\Users\a-halakh\qt5-experiments-2019-08-29.db")

In [3]:
import numpy as np
np.linspace(1,10000,100)

array([1.000e+00, 1.020e+02, 2.030e+02, 3.040e+02, 4.050e+02, 5.060e+02,
       6.070e+02, 7.080e+02, 8.090e+02, 9.100e+02, 1.011e+03, 1.112e+03,
       1.213e+03, 1.314e+03, 1.415e+03, 1.516e+03, 1.617e+03, 1.718e+03,
       1.819e+03, 1.920e+03, 2.021e+03, 2.122e+03, 2.223e+03, 2.324e+03,
       2.425e+03, 2.526e+03, 2.627e+03, 2.728e+03, 2.829e+03, 2.930e+03,
       3.031e+03, 3.132e+03, 3.233e+03, 3.334e+03, 3.435e+03, 3.536e+03,
       3.637e+03, 3.738e+03, 3.839e+03, 3.940e+03, 4.041e+03, 4.142e+03,
       4.243e+03, 4.344e+03, 4.445e+03, 4.546e+03, 4.647e+03, 4.748e+03,
       4.849e+03, 4.950e+03, 5.051e+03, 5.152e+03, 5.253e+03, 5.354e+03,
       5.455e+03, 5.556e+03, 5.657e+03, 5.758e+03, 5.859e+03, 5.960e+03,
       6.061e+03, 6.162e+03, 6.263e+03, 6.364e+03, 6.465e+03, 6.566e+03,
       6.667e+03, 6.768e+03, 6.869e+03, 6.970e+03, 7.071e+03, 7.172e+03,
       7.273e+03, 7.374e+03, 7.475e+03, 7.576e+03, 7.677e+03, 7.778e+03,
       7.879e+03, 7.980e+03, 8.081e+03, 8.182e+03, 

In [69]:
import numpy as np
maxn = 0 
for ii in np.linspace(1,10000,100):
    ds = load_by_id(ii)
    maxr = ds.number_of_results
    print(ii, maxr)
    if maxn <maxr:
        maxn = maxr

1.0 101
102.0 12
203.0 100
304.0 1600
405.0 1271
506.0 51
607.0 403
708.0 87
809.0 0
910.0 1
1011.0 1
1112.0 1
1213.0 1
1314.0 1
1415.0 1
1516.0 1
1617.0 1
1718.0 1
1819.0 1
1920.0 1
2021.0 1
2122.0 1
2223.0 1
2324.0 1
2425.0 1
2526.0 101
2627.0 101
2728.0 1
2829.0 1
2930.0 101
3031.0 161
3132.0 10611
3233.0 160
3334.0 301
3435.0 201
3536.0 200
3637.0 200
3738.0 201
3839.0 101
3940.0 101
4041.0 12
4142.0 182
4243.0 201
4344.0 101
4445.0 201
4546.0 28
4647.0 106
4748.0 301
4849.0 78
4950.0 401
5051.0 6
5152.0 11
5253.0 151
5354.0 151
5455.0 30351
5556.0 51
5657.0 1510
5758.0 1001
5859.0 101
5960.0 336
6061.0 151
6162.0 151
6263.0 66
6364.0 303
6465.0 151
6566.0 1
6667.0 110
6768.0 151
6869.0 1
6970.0 1
7071.0 1
7172.0 1
7273.0 0
7374.0 1
7475.0 1
7576.0 1
7677.0 1
7778.0 1
7879.0 1
7980.0 1
8081.0 80
8182.0 18
8283.0 1
8384.0 101
8485.0 101
8586.0 6960
8687.0 7061
8788.0 7162
8889.0 7263
8990.0 504
9091.0 9801
9192.0 601
9293.0 331
9394.0 2
9495.0 0
9596.0 2
9697.0 0
9798.0 4
9899.0 4
1

In [4]:
 ds = load_by_id(5455)


In [5]:
ds.paramspecs

{'qdac_p': ParamSpec('qdac_p', 'numeric', 'Plunger', 'V', inferred_from=[], depends_on=[]),
 'bias': ParamSpec('bias', 'numeric', 'Bias', 'V', inferred_from=[], depends_on=[]),
 'i_l': ParamSpec('i_l', 'numeric', 'I left', 'A', inferred_from=[], depends_on=['qdac_p', 'bias']),
 'g_ll': ParamSpec('g_ll', 'numeric', 'dI left / dV left', 'e²/h', inferred_from=[], depends_on=['qdac_p', 'bias']),
 'g_rl': ParamSpec('g_rl', 'numeric', 'dI right / dV left', 'e²/h', inferred_from=[], depends_on=['qdac_p', 'bias']),
 'i_r': ParamSpec('i_r', 'numeric', 'I right', 'A', inferred_from=[], depends_on=['qdac_p', 'bias']),
 'g_rr': ParamSpec('g_rr', 'numeric', 'dI right / dV right', 'e²/h', inferred_from=[], depends_on=['qdac_p', 'bias']),
 'g_lr': ParamSpec('g_lr', 'numeric', 'dI left / dV right', 'e²/h', inferred_from=[], depends_on=['qdac_p', 'bias'])}

In [9]:
%timeit ds.get_parameter_data('qdac_p', start=4000, end=4000)['qdac_p']

222 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [83]:
%timeit load_by_guid('aaaaaaaa-0000-0000-0000-01673807fd53')

635 ms ± 112 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 38.1 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

# Observation: Initial Connection taking longer time

In [6]:
from qcodes.dataset.data_set import get_DB_location,connect

In [18]:
conn = connect(get_DB_location())

In [21]:
conn.close()

In [8]:
%time load_by_run_spec(captured_run_id=1234, conn = conn)

Wall time: 7.21 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [9]:
%timeit load_by_id(run_id = 1234, conn = conn)

965 µs ± 39.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit load_by_guid(guid = 'aaaaaaaa-0000-0000-0000-01673807fd53', conn = conn)

1.02 ms ± 58.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
curs = conn.cursor()

ProgrammingError: Cannot operate on a closed database.

# Check if sql vaccum helps

In [11]:
%time conn.execute("VACUUM")

Wall time: 11min 12s


In [12]:
%time conn.execute("VACUUM")

Wall time: 9min 57s


In [13]:
%time conn.execute("VACUUM")

Wall time: 12min 10s


In [16]:
%time load_by_run_spec(captured_run_id=1234, conn = conn)

Wall time: 7.51 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [17]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 7.64 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [18]:
%timeit load_by_id(1234)

263 ms ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit load_by_guid('aaaaaaaa-0000-0000-0000-01673807fd53')

277 ms ± 49.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit load_by_id(run_id = 1234, conn = conn)

913 µs ± 67.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Doing "Vaccum" do increase the efficiency of output. Still it can not reduce less that few hundred ms because the first the new connection takes that much time. 

In [21]:
%timeit load_by_guid(guid = 'aaaaaaaa-0000-0000-0000-01673807fd53', conn = conn)

936 µs ± 102 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Profiling load_by_run_spec

In [87]:
import snakeviz

In [88]:
%load_ext snakeviz

In [24]:
%snakeviz load_by_run_spec(captured_run_id=1234)

 
*** Profile stats marshalled to file 'C:\\Users\\a-halakh\\AppData\\Local\\Temp\\tmp_um6vd8p'. 
Embedding SnakeViz in this document...


In [24]:
rows = curs.fetchall()

In [25]:
len(rows)

0

# Db Upgrade to index captured run id

In [25]:
from qcodes.dataset.sqlite.connection import atomic_transaction, atomic, transaction
from tqdm import tqdm
import sys

In [26]:

def perform_db_upgrade_8_to_9(conn) -> None:
    """
    Perform the upgrade from version 1 to version 2

    Add two indeces on the runs table, one for exp_id and one for GUID
    """

    sql = "SELECT name FROM sqlite_master WHERE type='table' AND name='runs'"
    cur = atomic_transaction(conn, sql)
    n_run_tables = len(cur.fetchall())

    pbar = tqdm(range(1), file=sys.stdout)
    pbar.set_description("Upgrading database; v1 -> v2")

    if n_run_tables == 1:
        _IX_runs_capture_id = """
                          CREATE INDEX
                          IF NOT EXISTS IX_runs_capture_id
                          ON runs (captured_run_id DESC)
                          """
        with atomic(conn) as conn:
            # iterate through the pbar for the sake of the side effect; it
            # prints that the database is being upgraded
            for _ in pbar:
                transaction(conn, _IX_runs_capture_id)
    else:
        raise RuntimeError(f"found {n_run_tables} runs tables expected 1")

In [27]:
perform_db_upgrade_8_to_9(conn) 

Upgrading database; v1 -> v2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.93s/it]


In [28]:
%time load_by_run_spec(captured_run_id=1234, conn = conn)

Wall time: 14 ms


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [35]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 352 ms


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [36]:
%time load_by_run_spec(captured_counter=1234)

Wall time: 7.35 s


results #2287@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [37]:
%timeit load_by_id(1234)

272 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit load_by_guid('aaaaaaaa-0000-0000-0000-01673807fd53')

233 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
